In [1]:
#referencia https://medium.com/towards-artificial-intelligence/multi-class-text-classification-using-pyspark-mllib-doc2vec-dbfcee5b39f2
import findspark
findspark.init() 
from pyspark.sql import SparkSession
import os


MAX_MEMORY = "8g"
spark = SparkSession.builder \
                    .master("local") \
                    .appName('va_repositorio')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .config("spark.jars", "E:\\va\\driverdb\\postgresql-42.2.5.jre6") \
                    .getOrCreate()

In [2]:
tabla="proyecto.va_repositorio_inicial"
df  = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/va") \
    .option("dbtable", tabla) \
    .option("user", "postgres") \
    .option("password", "david") \
    .option("driver", "org.postgresql.Driver") \
    .load()
df.printSchema()

root
 |-- periodo: string (nullable = true)
 |-- estu_mcpio_reside: string (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_estadocivil: string (nullable = true)
 |-- estu_etnia: string (nullable = true)
 |-- estu_limita_motriz: string (nullable = true)
 |-- estu_limita_invidente: string (nullable = true)
 |-- estu_limita_sordoconinterprete: string (nullable = true)
 |-- estu_limita_sordosininterprete: string (nullable = true)
 |-- estu_limita_sordoceguera: string (nullable = true)
 |-- estu_porcentajecreditosaprob: string (nullable = true)
 |-- estu_valormatriculauniversidad: string (nullable = true)
 |-- estu_pagomatriculabeca: string (nullable = true)
 |-- estu_pagomatriculacredito: string (nullable = true)
 |-- estu_pagomatriculapadres: string (nullable = true)
 |-- estu_pagomatriculapropio: string (nullable = true)
 |-- estu_tomo_cursopreparacion: string (nullable = true)
 |-- estu_cursodocentesies: string (nullable = true)
 |-- estu_cursoiesapoyoexterno: str

In [18]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lower, col
for c in df.columns: 
    df=(df
         .withColumn(c, regexp_replace(c, ' ', '_'))
         .withColumn(c, regexp_replace(c, ',', ''))
         .withColumn(c, regexp_replace(c, 'á', 'a'))
         .withColumn(c, regexp_replace(c, 'é', 'e'))
         .withColumn(c, regexp_replace(c, 'í', 'i'))
         .withColumn(c, regexp_replace(c, 'ó', 'o'))
         .withColumn(c, regexp_replace(c, 'ú', 'u'))
         .withColumn(c, regexp_replace(c, 'ñ', 'n'))
         .withColumn(c, lower(col(c))))

In [20]:
df=(df
         .withColumn("estu_mcpio_reside", regexp_replace("estu_mcpio_reside", 'ü', 'u')))

In [21]:
df.groupby("estu_mcpio_reside").count().show()

+--------------------+-----+
|   estu_mcpio_reside|count|
+--------------------+-----+
|san_andres_de_tumaco|  261|
|              ospina|   11|
|           guachucal|   31|
|               pasto| 7033|
|                cali|   28|
|               colon|   23|
|              cumbal|   33|
|           santacruz|    5|
|           samaniego|   20|
|               belen|    6|
|            santiago|    9|
|           taminango|   10|
|            riosucio|    1|
|             sandona|   29|
|           gualmatan|   20|
|   valle_del_guamuez|    4|
|          yacuanquer|   29|
|           tuquerres|   87|
|         providencia|    7|
|           san_pablo|    3|
+--------------------+-----+
only showing top 20 rows



In [22]:
df.write.jdbc(url="jdbc:postgresql://localhost:5432/va"
                  "?user=postgres&password=david",
              table="proyecto.va_repositorio_v2",
              mode="overwrite",
              properties={"driver": 'org.postgresql.Driver'})